In [1]:
import os, time
from collections import defaultdict
from keras.models import load_model
import pickle
import pandas as pd

c:\users\caimapo\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
resultados_path = os.path.join(os.getcwd(),'results')
model1 = load_model(os.path.join(resultados_path,'model_CuDNNGRU_.h5'))
#model2 = load_model(os.path.join(resultados_path,'model_CuDNNLSTM_.h5'))
#model3 = load_model(os.path.join(resultados_path,'model_RNN_.h5'))

In [3]:
df_test = pd.read_csv(os.path.join(os.getcwd(),'data','test_Q.csv'))

with open(os.path.join(os.getcwd(),'temp','Xtest_question.pickle'),'rb') as p_file:
    Xtest_question = pickle.load(p_file)
with open(os.path.join(os.getcwd(),'temp','vocab_answer.pickle'),'rb') as p_file:
    vocab_answer = pickle.load(p_file)
with open(os.path.join(os.getcwd(),'temp','vocab_question.pickle'),'rb') as p_file:
    vocab_question = pickle.load(p_file)
with open(os.path.join(os.getcwd(),'temp','vocabQ_indices.pickle'),'rb') as p_file:
    vocabQ_indices = pickle.load(p_file)
with open(os.path.join(os.getcwd(),'temp','indices_vocabA.pickle'),'rb') as p_file:
    indices_vocabA = pickle.load(p_file)

In [4]:
Xtest_question.shape

(11873, 60)

In [5]:
Xtest_question.reshape((11873,60,))

array([[41693, 15790,  8164, ...,     0,     0,     0],
       [33251, 28626, 28183, ...,     0,     0,     0],
       [ 4045, 16661, 27689, ...,     0,     0,     0],
       ...,
       [15790, 20168, 28183, ...,     0,     0,     0],
       [15790, 27235, 30533, ...,     0,     0,     0],
       [15790, 27235, 20168, ...,     0,     0,     0]])

In [6]:
Xtest_question.shape

(11873, 60)

In [7]:
vocab_question[39080]


'abell'

In [8]:
example = Xtest_question[1]
ex=example.reshape((1,60,))
print(example.shape)
print(ex.shape)

(60,)
(1, 60)


In [10]:
value = model1.predict(ex)

In [9]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 64)       2691776     input_1[0][0]                    
__________________________________________________________________________________________________
cu_dnngru_1 (CuDNNGRU)          (None, 60, 128)      74496       embedding_1[0][0]                
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 60, 47)       6063        cu_dnngru_1[0][0]                
__________________________________________________________________________________________________
permute_1 

In [52]:
value.shape

(1, 47, 47423)

In [11]:
print(value)

[[[1.19022043e-04 2.11404127e-04 8.35010269e-06 ... 5.91784953e-07
   2.23747043e-06 1.43329669e-06]
  [1.00577017e-02 1.17471545e-05 7.60854164e-06 ... 2.33514152e-06
   2.60590823e-05 2.58781711e-06]
  [3.02984565e-02 3.25669935e-05 3.36540666e-06 ... 1.32134601e-05
   2.37593595e-05 4.38707457e-06]
  ...
  [9.99964237e-01 1.86745619e-11 4.22336832e-10 ... 2.79410745e-10
   8.38289382e-10 3.23771127e-14]
  [9.99969006e-01 1.85884381e-11 3.16147081e-10 ... 2.89967939e-10
   8.21522850e-10 3.06602041e-14]
  [9.99966502e-01 1.86520712e-11 3.61135788e-10 ... 3.13238213e-10
   8.44432690e-10 3.08398902e-14]]]


In [54]:
val = value.reshape(value.shape[1:])
val.shape

(47, 47423)

In [56]:
import numpy as np

vak = np.sum(val,axis=1)
print(vak.shape)
print(vak[1])

(47,)
0.9999431


In [61]:
len(indices_vocabA)

47423

In [97]:
def predict_words(model, example):
    prediction = model.predict(example)
    prediction = prediction.reshape(prediction.shape[1:])
    palabra_elegida = np.array([])
    for p_palabra in prediction:
        p_palabra = p_palabra/np.sum(p_palabra)
        palabra_elegida = np.append(
            palabra_elegida, np.random.choice(p_palabra.shape[0], p=p_palabra))
    return palabra_elegida

In [109]:
def show_examples(model, indexs):

    for i in range(len(indexs)):
        #indexs = np.random.randint(0,len(Xtest_question))
        example = Xtest_question[indexs[i]]
        example = example.reshape((1,60,))
        indexes_answer = predict_words(model, example)

        question = df_test["question"][indexs[i]]
        print("Pregunta: ", question)
        answer = ""
        for index in indexes_answer:
            if indices_vocabA[index] == "#end":  # el final de la oracion
                continue
            else:
                answer += indices_vocabA[indexs[i]] + " "
        print("Respuesta: ", answer)
    print("Los ha predecido todos!")

In [112]:
tipos=np.random.randint(0,len(Xtest_question),size=10)

In [115]:
show_examples(model1,tipos)

Pregunta:  The university was a founding force behind what conference?
Respuesta:  khan khan khan 
Pregunta:  When was the Ottoman Caliphate abolished?
Respuesta:  top top 
Pregunta:  Which region of California is Palm Springs located in?
Respuesta:  Funnel Funnel Funnel Funnel Funnel Funnel Funnel 
Pregunta:  When was the first geological map of the entire earth created?
Respuesta:  pains 
Pregunta:  What did Donald Davies develop in the late 1950s?
Respuesta:  trophies trophies trophies trophies 
Pregunta:  What club won 118 tournaments and 15 national championships?
Respuesta:  overthrow overthrow overthrow overthrow overthrow 
Pregunta:  Who had a main goal of attracting the wealthy to Harvard?
Respuesta:  glamour glamour glamour 
Pregunta:  A vertical force pointing northeast can be split into how many forces?
Respuesta:  Site Site Site Site 
Pregunta:  Point Conception is an example of a landmark among what boundary of southern California?
Respuesta:  Luigi Luigi 
Pregunta:  How 

In [116]:
show_examples(model2,tipos)

Pregunta:  The university was a founding force behind what conference?
Respuesta:  khan khan khan khan 
Pregunta:  When was the Ottoman Caliphate abolished?
Respuesta:  top top 
Pregunta:  Which region of California is Palm Springs located in?
Respuesta:  Funnel Funnel Funnel Funnel Funnel Funnel Funnel Funnel Funnel Funnel Funnel Funnel 
Pregunta:  When was the first geological map of the entire earth created?
Respuesta:  pains pains 
Pregunta:  What did Donald Davies develop in the late 1950s?
Respuesta:  trophies trophies 
Pregunta:  What club won 118 tournaments and 15 national championships?
Respuesta:  overthrow overthrow overthrow 
Pregunta:  Who had a main goal of attracting the wealthy to Harvard?
Respuesta:  glamour 
Pregunta:  A vertical force pointing northeast can be split into how many forces?
Respuesta:  Site Site Site 
Pregunta:  Point Conception is an example of a landmark among what boundary of southern California?
Respuesta:  Luigi 
Pregunta:  How much of the lake co

In [117]:
show_examples(model3,tipos)

Pregunta:  The university was a founding force behind what conference?
Respuesta:  khan khan 
Pregunta:  When was the Ottoman Caliphate abolished?
Respuesta:  top top 
Pregunta:  Which region of California is Palm Springs located in?
Respuesta:  Funnel Funnel Funnel Funnel Funnel 
Pregunta:  When was the first geological map of the entire earth created?
Respuesta:  pains 
Pregunta:  What did Donald Davies develop in the late 1950s?
Respuesta:  trophies trophies 
Pregunta:  What club won 118 tournaments and 15 national championships?
Respuesta:  overthrow 
Pregunta:  Who had a main goal of attracting the wealthy to Harvard?
Respuesta:  glamour 
Pregunta:  A vertical force pointing northeast can be split into how many forces?
Respuesta:  Site Site Site Site Site Site Site Site 
Pregunta:  Point Conception is an example of a landmark among what boundary of southern California?
Respuesta:  Luigi Luigi 
Pregunta:  How much of the lake connecting with the Rhine can you see from the German is